In [275]:
import os
import sys
import numpy as np
import cPickle
import random
import math
import h5py
from collections import defaultdict as ddu
import tensorflow as tf
from tensorflow.python.ops.init_ops import glorot_uniform_initializer, RandomUniform, RandomNormal

from keras import layers
from keras.models import Sequential
from keras.layers import Input, Dense, Activation, BatchNormalization, Flatten
from keras.layers.embeddings import Embedding
from keras.models import Model
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences



np.random.seed(1)
random.seed(1)

In [431]:
# Initialize parameters
DATASET = 'citeseer'

embedding_size = 50
learning_rate = 0.1
batch_size = 200
neg_samp = 0
model_file = 'trans.model'

window_size = 3
path_size = 10

g_batch_size = 200
g_learning_rate = 0.01
g_sample_size = 100

use_feature = True
update_emb = False
layer_loss =  False

In [505]:
# Load the dataset
NAMES = ['x', 'y', 'tx', 'ty', 'graph']
OBJECTS = []
for i in range(len(NAMES)):
    OBJECTS.append(cPickle.load(open("/hdd2/graph_embedding/planetoid/data/trans.{}.{}".format(DATASET, NAMES[i]))))
x, y, tx, ty, graph = tuple(OBJECTS)

In [506]:
def comp_iter(iter):
    """an auxiliary function used for computing the number of iterations given the argument iter.
    iter can either be an int or a float.
    """
    if iter >= 1:
        return iter
    return 1 if random.random() < iter else 0

In [507]:
def create_placeholders():
    """
    Creates the placeholders for the tensorflow session.
    
    Arguments:
    n_x -- scalar, height of an input image
    n_W0 -- scalar, width of an input image
    n_C0 -- scalar, number of channels of the input
    n_y -- scalar, number of classes
        
    Returns:
    X -- placeholder for the data input, of shape [n_x, None] and dtype "float"
    Y -- placeholder for the input labels, of shape [None, n_y] and dtype "float"
    """

    
    x_sym = tf.placeholder(tf.float32, shape = [None, 3703], name = 'x')
    y_sym = tf.placeholder(tf.int32, shape = [None, 6], name = 'y')
    g_sym = tf.placeholder(tf.int32, shape = [None, ], name = 'g')
    gy_sym = tf.placeholder(tf.int32, shape = [None, 1], name = 'gy')
    ind_sym = tf.placeholder(tf.float32, shape = [None, 50], name = 'ind')
    
    
    return x_sym, y_sym, g_sym, gy_sym, ind_sym

In [326]:
# # def build():
# """
# Builds the model.
# """
# np.random.seed(1)
# random.seed(1)
# tf.random_normal_initializer(seed = 1)
# num_sampled = 64
# num_ver = max(graph.keys()) + 1
# vocabulary_size = num_ver

# # graph = tf.Graph()

# # with graph.as_default(), tf.device('/cpu:0'):
    
# x_sym, y_sym, g_sym, gy_sym, ind_sym = create_placeholders()



# # word embedding
# tf.random_normal_initializer(seed = 1)
# tf.set_random_seed(1)
# embeddings = tf.Variable(
#     tf.random_normal([vocabulary_size, embedding_size], mean = 0.0, stddev = 0.01))
# softmax_weights = tf.Variable(
#     tf.random_normal([vocabulary_size, embedding_size], mean = 0.0, stddev = 0.01))
# softmax_biases = tf.Variable(tf.zeros([vocabulary_size]))
# l_emd_f = tf.nn.embedding_lookup(embeddings, g_sym)

# ##

# l_x_hid = tf.layers.dense(inputs = x_sym, units = y.shape[1], activation = tf.nn.softmax)
# if use_feature:
#     l_emd_z = tf.layers.dense(inputs = ind_sym, units = y.shape[1], activation = tf.nn.softmax, 
#                               kernel_initializer=glorot_uniform_initializer(seed=1))
#     l_f = tf.concat([l_x_hid, l_emd_z], axis = 1)
#     l_y = tf.layers.dense(inputs = l_f, units = y.shape[1], activation = tf.nn.softmax)
# else:
#     l_y = tf.layers.dense(inputs = ind_sym, units = y.shape[1], activation = tf.nn.softmax)

l_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = l_y, labels = y_sym)) 

# if layer_loss and use_feature:
#     l_loss += tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = l_x_hid, labels = y_sym))
#     l_loss += tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = l_emd_z, labels = y_sym))

# # if neg_samp == 0:
# #     pass
# # else:

# # g_loss = tf.reduce_mean(
# #     tf.nn.sampled_softmax_loss(weights=softmax_weights, biases=softmax_biases, 
# #                                inputs = l_emd_f, labels = gy_sym, 
# #                                num_sampled = vocabulary_size, 
# #                                num_classes = vocabulary_size))
    

# optimizer = tf.train.AdamOptimizer(learning_rate).minimize(l_loss)

# # g_optimizer = tf.train.AdamOptimizer(g_learning_rate).minimize(g_loss)

In [201]:
def gen_train_inst():
    """generator for batches for classification loss.
    """
    np.random.seed(1)
    random.seed(1)
    while True:
        ind = np.array(np.random.permutation(x.shape[0]), dtype = np.int32)
        i = 0
        while i < ind.shape[0]:
            j = min(ind.shape[0], i + batch_size)
            yield x[ind[i: j]], y[ind[i: j]], ind[i: j]
            i = j

In [202]:
def gen_label_graph():
    """generator for batches for label context loss.
    """
    np.random.seed(1)
    random.seed(1)
    labels, label2inst, not_label = [], dd(list), dd(list)
    for i in range(x.shape[0]):
        flag = False
        for j in range(y.shape[1]):
            if y[i, j] == 1 and not flag:
                labels.append(j)
                label2inst[j].append(i)
                flag = True
            elif y[i, j] == 0:
                not_label[j].append(i)

    while True:
        g, gy = [], []
        for _ in range(g_sample_size):
            x1 = random.randint(0, x.shape[0] - 1)
            label = labels[x1]
            if len(label2inst) == 1: continue
            x2 = random.choice(label2inst[label])
            g.append([x1, x2])
            gy.append(1.0)
#             for _ in range(neg_samp):
#                 g.append([x1, random.choice(not_label[label])])
#                 gy.append( - 1.0)
        yield np.array(g, dtype = np.int32), np.array(gy, dtype = np.float32)

In [203]:
def gen_graph():
    """generator for batches for graph context loss.
    """
    np.random.seed(1)
    random.seed(1)
    num_ver = max(graph.keys()) + 1

    while True:
        ind = np.random.permutation(num_ver)
        i = 0
        while i < ind.shape[0]:
            g, gy = [], []
            j = min(ind.shape[0], i + g_batch_size)
            for k in ind[i: j]:
                if len(graph[k]) == 0: continue
                path = [k]
                for _ in range(path_size):
                    path.append(random.choice(graph[path[-1]]))
                for l in range(len(path)):
                    for m in range(l - window_size, l + window_size + 1):
                        if m < 0 or m >= len(path): continue
                        g.append([path[l], path[m]])
                        gy.append(1.0)
#                         for _ in range(neg_samp):
#                             # if the random number euqals to path[m], the it creates noise!
#                             g.append([path[l], random.randint(0, num_ver - 1)])
#                             gy.append(- 1.0)
            yield np.array(g, dtype = np.int32), np.array(gy, dtype = np.float32)
            i = j

In [502]:
max_iter = 1
iter_graph = 0
iter_inst = 1
iter_label = 0
del l_x_hid

In [503]:
# def build():
"""
Builds the model.
"""
tf.reset_default_graph()
np.random.seed(1)
random.seed(1)
tf.random_normal_initializer(seed = 1)
num_sampled = 64
num_ver = max(graph.keys()) + 1
vocabulary_size = num_ver

# graph = tf.Graph()

# with graph.as_default(), tf.device('/cpu:0'):
    
x_sym, y_sym, g_sym, gy_sym, ind_sym = create_placeholders()



# word embedding
tf.random_normal_initializer(seed = 1)
tf.set_random_seed(1)
embeddings = tf.Variable(
    tf.fill([vocabulary_size, embedding_size], 0.6))
softmax_weights = tf.Variable(
    tf.fill([vocabulary_size, embedding_size], 0.5))

softmax_biases = tf.Variable(tf.zeros([vocabulary_size]))
l_emd_f = tf.nn.embedding_lookup(embeddings, g_sym)

##
# del l_x_hid
l_x_hid = tf.layers.dense(inputs = x_sym, units = y.shape[1], activation = tf.nn.softmax, name = 'l_x_hid_layer',
                         kernel_initializer=tf.constant_initializer(0.2),)
# if use_feature:
l_emd_z = tf.layers.dense(inputs = ind_sym, units = y.shape[1], activation = tf.nn.softmax, 
                          kernel_initializer=tf.constant_initializer(0.4))
l_f = tf.concat([l_x_hid, l_emd_z], axis = 1)
l_y = tf.layers.dense(inputs = l_f, units = y.shape[1], activation = tf.nn.softmax, name='l_y_layer',
                     kernel_initializer= tf.constant_initializer(0.3))
# else:
#     l_y = tf.layers.dense(inputs = ind_sym, units = y.shape[1], activation = tf.nn.softmax, 
#                       kernel_initializer= tf.constant_initializer(0.4))

l_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = l_y, labels = y_sym))

# if layer_loss and use_feature:
#     l_loss += tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = l_x_hid, labels = y_sym))
#     l_loss += tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = l_emd_z, labels = y_sym))

# if neg_samp == 0:
#     pass
# else:

# g_loss = tf.reduce_mean(
#     tf.nn.sampled_softmax_loss(weights=softmax_weights, biases=softmax_biases, 
#                                inputs = l_emd_f, labels = gy_sym, 
#                                num_sampled = vocabulary_size, 
#                                num_classes = vocabulary_size))
    

optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(l_loss)

# g_optimizer = tf.train.AdamOptimizer(g_learning_rate).minimize(g_loss)

In [504]:
init_iter_label = 2
init_iter_graph = 2
inst_generator = gen_train_inst()
graph_generator = gen_graph()
# Generates pairs with the same label (1) or different labels (-1)
label_generator = gen_label_graph()

# init_train
with tf.Session() as session:
    tf.global_variables_initializer().run()
#     print('init_iter_label')
#     for i in range(init_iter_label):
#         gx, gy = next(label_generator)
# #         print(gx)
#         feed_dict={g_sym: gx[:, 0], gy_sym: gx[:, 1].reshape(gx.shape[0],1)}
#         _, l = session.run([g_optimizer, g_loss], feed_dict=feed_dict)
#         print 'iter label', i, l
#     print('init_iter_graph')
#     for i in range(init_iter_graph):
#         gx, gy = next(graph_generator)
#         feed_dict={g_sym: gx[:, 0], gy_sym: gx[:, 1].reshape(gx.shape[0],1)}
#         _, l = session.run([g_optimizer, g_loss], feed_dict=feed_dict)
#         print 'iter graph', i, l

    iter_cnt = 0
    while True:
        for _ in range(max_iter):
#             for _ in range(comp_iter(iter_graph)):
#                 gx, gy = next(graph_generator)
#                 feed_dict={g_sym: gx[:, 0], gy_sym: gx[:, 1].reshape(gx.shape[0],1)}
#                 _, l = session.run([g_optimizer, g_loss], feed_dict=feed_dict)

            for i in range(5):
                print('=================== iter: %d ==============' %i)
                xs, ys, indexs = next(inst_generator)
#                 gx, gy = next(label_generator)
                xs = xs.toarray()
                inds = embeddings.eval()[indexs]
#                 print('indexs:')
#                 print(indexs)
#                 print('softmax_biases:')
#                 print(softmax_biases.eval())
#                 print('softmax_weights:')
#                 print(softmax_weights.eval()[0])
#                 print('embeddings:')
#                 print(embeddings.eval()[0])
#                 print('inds[0]:')
#                 print(inds[0])
                
#                 tmp_emd = embeddings.eval()
                
#                 print(type(inds))
#                 sys.exit(0)
                feed_dict={x_sym: xs, y_sym: ys, ind_sym: inds}
                _, l, res_l_y, res_l_f, res_l_x_hid_w = session.run([optimizer, l_loss, l_y,l_f, l_x_hid], feed_dict=feed_dict)
                
                l_y_w = tf.get_default_graph().get_tensor_by_name(os.path.split(l_y.name)[0] + '/kernel:0')
                l_x_hid_w = tf.get_default_graph().get_tensor_by_name(os.path.split(l_x_hid.name)[0] + '/kernel:0')
                print('xs:')
                print(xs)
                print('l_x_hid_w:')
                print(l_x_hid_w.eval())
                
                print('l_y_w:')
                print(l_y_w.eval())
                print('res_l_y:')
                print(res_l_y)
#                 print(res_ind_sym)
                print('l_f:')
                print(res_l_f[0])
#                 sys.exit(0)
#                 print('iter:%d inst: %.4f' %(i, l))
            sys.exit(0)

#             for _ in range(comp_iter(iter_label)):
#                 gx, gy = next(label_generator)
#                 feed_dict={g_sym: gx[:, 0], gy_sym: gx[:, 1].reshape(gx.shape[0],1)}
#                 _, l = session.run([g_optimizer, g_loss], feed_dict=feed_dict)


        predict_y = tf.arg_max(l_y, 1)
        correct_prediction = tf.equal(predict_y, tf.arg_max(y_sym, 1))

        # Calculate accuracy
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, 'float'))
    #     print(accuracy)
#         print('number of iteration: %d', iter_cnt)
        iter_cnt += 1
        trn_ind = embeddings.eval()[0:120]
        train_accuracy = accuracy.eval({x_sym: x.toarray(), y_sym: y, ind_sym: trn_ind})
#         print('Train Accuracy: %.4f', train_accuracy)
        txs = tx.toarray()
        t_index = np.arange(x.shape[0], x.shape[0] + tx.shape[0], dtype = np.int32)
        t_index_emd = embeddings.eval()[t_index]
        test_accuracy = accuracy.eval({x_sym: txs, y_sym: ty, ind_sym: t_index_emd})
        print('iter: %d, Tst Acc: %.4f, Trn Acc: %.4f' %(iter_cnt, test_accuracy, train_accuracy))


=================== iter: 0 ==============
xs:
[[ 0.  1.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
l_x_hid_w:
[[ 0.2  0.2  0.2  0.2  0.2  0.2]
 [ 0.2  0.2  0.2  0.2  0.2  0.2]
 [ 0.2  0.2  0.2  0.2  0.2  0.2]
 ..., 
 [ 0.2  0.2  0.2  0.2  0.2  0.2]
 [ 0.2  0.2  0.2  0.2  0.2  0.2]
 [ 0.2  0.2  0.2  0.2  0.2  0.2]]
l_y_w:
[[ 0.30000001  0.30000001  0.30000001  0.30000001  0.30000001  0.30000001]
 [ 0.30000001  0.30000001  0.30000001  0.30000001  0.30000001  0.30000001]
 [ 0.30000001  0.30000001  0.30000001  0.30000001  0.30000001  0.30000001]
 [ 0.30000001  0.30000001  0.30000001  0.30000001  0.30000001  0.30000001]
 [ 0.30000001  0.30000001  0.30000001  0.30000001  0.30000001  0.30000001]
 [ 0.30000001  0.30000001  0.30000001  0.30000001  0.30000001  0.30000001]
 [ 0.30000001  0.30000001  0.30000001  0.30000001  0.30000001  0.30000001]
 [ 0

SystemExit: 0

In [482]:
res_l_y.shape

(120, 6)

In [443]:
os.path.split(l_x_hid.name)

(u'dense', u'Softmax:0')

In [421]:
l_y_w.shape

TensorShape([Dimension(50), Dimension(6)])

In [353]:
# with tf.Session() as session:
# #     tf.global_variables_initializer().run()
#     feed_dict={x_sym: xs, y_sym: ys}
#     ll, l = session.run([optimizer, loss], feed_dict=feed_dict)

In [319]:
xs[0]

array([ 0.,  0.,  0., ...,  0.,  0.,  0.], dtype=float32)

In [320]:
txs[0]

array([ 0.,  0.,  0., ...,  0.,  0.,  0.], dtype=float32)

In [321]:
ys[0]

array([0, 0, 0, 0, 1, 0], dtype=int32)

In [324]:
t_index = np.arange(x.shape[0], x.shape[0] + tx.shape[0], dtype = np.int32)

In [265]:
tf.get_default_graph().get_tensor_by_name(os.path.split(l_y.name)[0] + '/kernel:0')

KeyError: "The name 'dense_73/kernel:0' refers to a Tensor which does not exist. The operation, 'dense_73/kernel', does not exist in the graph."

In [302]:
# os.path.split(l_y.name)[0] + '/kernel:0'

tf.get_default_graph().get_tensor_by_name(os.path.split(l_x_hid.name)[0] + '/kernel:0')

<tf.Tensor 'dense_94/kernel:0' shape=(3703, 6) dtype=float32_ref>

In [472]:
tf.get_default_graph().get_tensor_by_name(os.path.split(l_x_hid.name)[0] + '/kernel:0')

<tf.Tensor 'l_x_hid_layer/kernel:0' shape=(3703, 6) dtype=float32_ref>

ValueError: Cannot evaluate tensor using `eval()`: No default session is registered. Use `with sess.as_default()` or pass an explicit session to `eval(session=sess)`

In [475]:
os.path.split(l_x_hid.name)[0]

u'l_x_hid_layer'